In [5]:
import torch
import torchvision.transforms as tvt
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import ConcatDataset, DataLoader
from torchmetrics import AUROC  # additional dependency
from torchvision.datasets import CIFAR10

from oodtk import NegativeEnergy, Softmax
from oodtk.dataset.img import Textures, CIFAR10C, CIFAR10P
from oodtk.model import WideResNet
from oodtk.utils import is_unknown

In [6]:
torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

# setup data
dataset_train = CIFAR10(root="data", train=True, download=True, transform=trans)
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test = Textures(root="data", download=True, transform=trans)
dataset_test = ConcatDataset([dataset_in_test, dataset_out_test])
train_loader = DataLoader(dataset_train, batch_size=128)
test_loader = DataLoader(dataset_test, batch_size=128)

Files already downloaded and verified


In [4]:
model = WideResNet.from_pretrained("oe-cifar10-tune", num_classes=10)
opti = Adam(model.parameters())
criterion = CrossEntropyLoss()
model = model.cuda()

In [5]:
# model.train()
#
# # start training
# for n, batch in enumerate(train_loader):
#     x, y = batch
#     x = x.cuda()
#     y = y.cuda()
#
#     logits = model(x)
#     loss = criterion(logits, y)
#     opti.zero_grad()
#     loss.backward()
#     opti.step()


In [6]:
# create some methods
energy = NegativeEnergy(model).cuda()
softmax = Softmax(model).cuda()

# evaluate
auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

/home/ki/.local/share/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


tensor(0.9775, device='cuda:0')
tensor(0.9708, device='cuda:0')


/home/ki/.local/share/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)


In [7]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test = CIFAR10C(root="data", download=True, transform=trans)
dataset_test = dataset_in_test +  dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

TypeError: __init__() missing 1 required positional argument: 'subset'

In [3]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test = CIFAR10P(root="data", download=True, transform=trans)
dataset_test = dataset_in_test +  dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

KeyboardInterrupt: 

In [3]:
dataset_out_test = CIFAR10C(root="data", subset="all", download=True, transform=trans)

In [7]:

dataset_out_test = CIFAR10P(root="data", download=True, transform=trans)

0it [00:00, ?it/s]

Extracting data/CIFAR-10-P.tar to data
